# Reading random Terrain data
This is an experiment to demonstrate the value of putting ScienceBase item on S3 in a cloud-optimized format. 

I took the 230GB `.tar.bz2` file on S3 from https://www.sciencebase.gov/catalog/item/5af335b2e4b0da30c1b26dab, converted it to 240GB Cloud Optimized GeoTIFF (COG), and put it back on S3. Here's a simple example of reading/visualizing one of the COG overlays using xarray, rasterio and holoviz tools.

In [ ]:
import rasterio
import xarray as xr
import hvplot.xarray
import geoviews as gv
import rioxarray as rx
import os

DEM data is stored as COG on S3

In [ ]:
os.environ["AWS_REQUEST_PAYER"] = "requester" 

In [ ]:
cog_urls = {}

URL below was obtained by browsing [this page](http://prd-tnm.s3.amazonaws.com/index.html?prefix=StagedProducts/Elevation/1/TIFF/n31w100/)

In [ ]:
cog_urls['random']='https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/1/TIFF/n31w100/USGS_1_n31w100.tif'

In [ ]:
cog_urls['USVI']="https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/1/TIFF/n19w065/USGS_1_n19w065.tif"

In [ ]:
da = xr.open_rasterio(cog_urls['USVI']);

In [ ]:
print(da[0,0,0].values)

Check out the overviews:

In [ ]:
src = rasterio.open(cog_url, 'r')
[src.overviews(i) for i in src.indexes]

Pick an overview and read directly from that:

In [ ]:
#da = rx.open_rasterio(cog_url, masked=True, overview_level=7)

Use rioxarray to get CRS

In [ ]:
crs = da.rio.crs.to_epsg()
crs

In [ ]:
da[0,0,0]

Plot up the bathy with a tile service basemap

In [ ]:
da = da.where(da>0)[0,:,:]

In [ ]:
da

In [ ]:
gv.tile_sources.OSM  * da.hvplot.quadmesh(rasterize=True,geo=True,colormap='viridis').opts(alpha=0.7)

Ocean bathy are values less below sea level from 1st band:

Try hill shaded relief using datashader's [geoprocessing routines](http://datashader.org/user_guide/8_Geography.html) 

In [ ]:
import datashader.geo as dsgeo

In [ ]:
ocean_hs = dsgeo.hillshade(ocean)

In [ ]:
gv.tile_sources.OSM  * ocean.copy(data=ocean_hs.data).hvplot.image(rasterize=True, crs=crs, colormap='gray')